PM10 Nasa processing data

In [28]:
import netCDF4
import numpy as np
import pandas as pd
print(netCDF4.__version__)
import geopandas as gpd
from shapely.geometry import Point
import fiona

1.7.2


In [29]:
from netCDF4 import Dataset

In [30]:
file_path = 'pm10-nasa/2020/MERRA2_400.tavgM_2d_aer_Nx.202001.nc4'

layer_number = 71  # Python indexing starts from 0, so layer 72 is index 71
air_density_value = 1.225  # kg/m³

try:
    with Dataset(file_path, 'r') as nc_file:
        # Check if the total mass variables exist
        required_totals = ['SO4SMASS', 'BCSMASS', 'OCCMASS', 'DUSMASS', 'SSSMASS']
        if not all(var in nc_file.variables for var in required_totals):
            missing_totals = [var for var in required_totals if var not in nc_file.variables]
            print(f"Error: The following required total mass variables are missing: {missing_totals}")
        else:
            # Extract the necessary 2D variables
            so4 = nc_file.variables['SO4SMASS'][:]
            bc = nc_file.variables['BCSMASS'][:]
            oc = nc_file.variables['OCCMASS'][:]
            du_total = nc_file.variables['DUSMASS'][:]
            ss_total = nc_file.variables['SSSMASS'][:]

            # Assuming BC and OC are split into hydrophobic and hydrophilic
            bc_phobic = bc * 0.5
            bc_philic = bc * 0.5
            oc_phobic = oc * 0.5
            oc_philic = oc * 0.5

            # Distribute total dust mass based on the PM10 equation coefficients
            total_du_coeff = 1 + 1 + 1 + 0.74
            du001_approx = du_total * (1 / total_du_coeff)
            du002_approx = du_total * (1 / total_du_coeff)
            du003_approx = du_total * (1 / total_du_coeff)
            du004_approx = du_total * (0.74 / total_du_coeff)

            # Distribute total sea salt mass evenly
            ss001_approx = ss_total * 0.25
            ss002_approx = ss_total * 0.25
            ss003_approx = ss_total * 0.25
            ss004_approx = ss_total * 0.25

            # Calculate PM10 concentration
            pm10 = (1.375 * so4 + bc_phobic + bc_philic + oc_phobic + oc_philic +
                    du001_approx + du002_approx + du003_approx + 0.74 * du004_approx +
                    ss001_approx + ss002_approx + ss003_approx + ss004_approx) * air_density_value

            print("Approximate PM10 Concentrations (2D field):")
            print(pm10)

except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'")
except Exception as e:
    print(f"An error occurred: {e}")

Approximate PM10 Concentrations (2D field):
[[[3.80578860e-06 3.80578860e-06 3.80578860e-06 ... 3.80578860e-06
   3.80578860e-06 3.80578860e-06]
  [3.79316120e-06 3.79406740e-06 3.79406740e-06 ... 3.79163120e-06
   3.79163120e-06 3.79163120e-06]
  [3.76665002e-06 3.76665002e-06 3.76665002e-06 ... 3.76665002e-06
   3.76665002e-06 3.76665002e-06]
  ...
  [9.38156420e-07 9.38156420e-07 9.38156420e-07 ... 9.38156420e-07
   9.38156420e-07 9.38156420e-07]
  [9.36620252e-07 9.36544631e-07 9.36544631e-07 ... 9.36790692e-07
   9.36790692e-07 9.36790692e-07]
  [9.35406075e-07 9.35406075e-07 9.35406075e-07 ... 9.35406075e-07
   9.35406075e-07 9.35406075e-07]]]


In [31]:
try:
    with fiona.Env(SHAPE_RESTORE_SHX='YES'):
        # Then proceed with reading your shapefile using geopandas
        shapefile_path = 'illinois_shape/IL_BNDY_County_Py.shp'  # Make sure this is correct
        illinois_counties = gpd.read_file(shapefile_path)
        # ... your code to process the shapefile ...
except fiona.errors.DriverError as e:
    print(f"Error reading shapefile with Fiona: {e}")
except FileNotFoundError:
    print(f"Error: Shapefile not found at {shapefile_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [32]:
illinois_counties = gpd.read_file(shapefile_path)
print("Columns in the shapefile:")
print(illinois_counties.columns)

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


In [33]:
try:
    with fiona.Env(SHAPE_RESTORE_SHX='YES'):
        # Then proceed with reading your shapefile using geopandas
        shapefile_path = 'illinois_shape/IL_BNDY_County_Py.shp'  # Make sure this is correct
        illinois_counties = gpd.read_file(shapefile_path)
        # ... your code to process the shapefile ...
except fiona.errors.DriverError as e:
    print(f"Error reading shapefile with Fiona: {e}")
except FileNotFoundError:
    print(f"Error: Shapefile not found at {shapefile_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

function to output the monthly concentrations

In [34]:
def calculate_average_pm10(file_path, shapefile_path):
    """
    Calculates the average PM10 concentration for each county (or sub-county region) in a shapefile,
    using data from a NetCDF file.
    """
    air_density_value = 1.225  # kg/m³

    # Define approximate Illinois latitude and longitude boundaries
    min_lat = 36.9
    max_lat = 42.5
    min_lon = -91.5
    max_lon = -87.5
    try:
        with netCDF4.Dataset(file_path, 'r') as nc_file:
            latitudes = nc_file.variables['lat'][:]
            longitudes = nc_file.variables['lon'][:]

            # Find indices of grid points within Illinois latitude and longitude bounds
            lat_indices = np.where((latitudes >= min_lat) & (latitudes <= max_lat))[0]
            lon_indices = np.where((longitudes >= min_lon) & (longitudes <= max_lon))[0]

            # Extract Illinois latitude and longitude coordinates
            illinois_lats = latitudes[lat_indices]
            illinois_lons = longitudes[lon_indices]

            # Extract the 2D aerosol mass mixing ratio variables over the Illinois region
            def extract_illinois_data(var_name):
                if var_name in nc_file.variables:
                    var_data = nc_file.variables[var_name][0, lat_indices, lon_indices]  # Assuming time is the first dimension
                    return var_data
                else:
                    print(f"Warning: Variable '{var_name}' not found.")
                    return None

            so4_il = extract_illinois_data('SO4SMASS')
            bc_il = extract_illinois_data('BCSMASS')
            oc_il = extract_illinois_data('OCCMASS')
            du_il = extract_illinois_data('DUSMASS')
            ss_il = extract_illinois_data('SSSMASS')

            if so4_il is not None and bc_il is not None and oc_il is not None and du_il is not None and ss_il is not None:
                # Assuming BC and OC are split into hydrophobic and hydrophilic
                bc_phobic_il = bc_il * 0.5
                bc_philic_il = bc_il * 0.5
                oc_phobic_il = oc_il * 0.5
                oc_philic_il = oc_il * 0.5

                # Distribute total dust mass based on the PM10 equation coefficients
                total_du_coeff = 1 + 1 + 1 + 0.74
                du001_approx_il = du_il * (1 / total_du_coeff)
                du002_approx_il = du_il * (1 / total_du_coeff)
                du003_approx_il = du_il * (1 / total_du_coeff)
                du004_approx_il = du_il * (0.74 / total_du_coeff)

                # Distribute total sea salt mass evenly
                ss001_approx_il = ss_il * 0.25
                ss002_approx_il = ss_il * 0.25
                ss003_approx_il = ss_il * 0.25
                ss004_approx_il = ss_il * 0.25

                # Calculate PM10 concentration over Illinois
                pm10_il = (1.375 * so4_il + bc_phobic_il + bc_philic_il + oc_phobic_il + oc_philic_il +
                           du001_approx_il + du002_approx_il + du003_approx_il + 0.74 * du004_approx_il +
                           ss001_approx_il + ss002_approx_il + ss003_approx_il + ss004_approx_il) * air_density_value

                # Load Illinois counties shapefile
                illinois_counties = gpd.read_file(shapefile_path)

                # Print the columns to help identify the correct one for the county/region names
                print("Columns in the shapefile:")
                print(illinois_counties.columns)

                # Reproject the shapefile to the CRS of the NetCDF data (assuming WGS84 - EPSG:4326) if necessary
                if illinois_counties.crs != 'epsg:4326':  # Check if the CRS is already WGS84
                    illinois_counties = illinois_counties.to_crs(epsg=4326)

                county_avg_pm10 = {}

                # Iterate through each county
                for index, county in illinois_counties.iterrows():
                    county_name = county['COUNTY_NAM']  # <--- Change this if 'NAME' is not correct
                    county_polygon = county['geometry']
                    county_pm10_values = []

                    # Iterate through the Illinois grid points
                    for lat_idx, lat in enumerate(illinois_lats):
                        for lon_idx, lon in enumerate(illinois_lons):
                            point = Point(lon, lat)
                            if county_polygon.contains(point):
                                # Use lat_idx and lon_idx directly to access the PM10 data for Illinois
                                if pm10_il.ndim == 2 and lat_idx < pm10_il.shape[0] and lon_idx < pm10_il.shape[1]:
                                    county_pm10_values.append(pm10_il[lat_idx, lon_idx])
                                elif pm10_il.ndim == 3 and 0 < pm10_il.shape[1] and lat_idx < pm10_il.shape[1] and lon_idx < pm10_il.shape[2]:
                                    county_pm10_values.append(pm10_il[0, lat_idx, lon_idx])
                                else:
                                    print(
                                        f"Warning: Index out of bounds for PM10 data at lat_idx={lat_idx}, lon_idx={lon_idx}")

                    if county_pm10_values:
                        avg_pm10 = np.mean(county_pm10_values)
                        county_avg_pm10[county_name] = avg_pm10
                    else:
                        county_avg_pm10[county_name] = 0  # Use np.nan for missing data

                # Create a Pandas DataFrame from the county_avg_pm10 dictionary
                df = pd.DataFrame(list(county_avg_pm10.items()), columns=['County', 'Average PM10'])
                return df

            else:
                return None # Return None if the aerosol data is not available
    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}' or '{shapefile_path}'")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [35]:
file_path_2020_01 = 'pm10-nasa/2020/MERRA2_400.tavgM_2d_aer_Nx.202001.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2020_01 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2020_01 = calculate_average_pm10(file_path_2020_01, shapefile_path_2020_01)
pm10_df_2020_01

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000003
3,WILL,0.000004
4,LASALLE,0.000003
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000004
100,UNION,0.000004


In [36]:
file_path_2020_02 = 'pm10-nasa/2020/MERRA2_400.tavgM_2d_aer_Nx.202002.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2020_02 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2020_02 = calculate_average_pm10(file_path_2020_02, shapefile_path_2020_02)
pm10_df_2020_02

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000003
3,WILL,0.000003
4,LASALLE,0.000003
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000004
100,UNION,0.000003


In [37]:
file_path_2020_03 = 'pm10-nasa/2020/MERRA2_400.tavgM_2d_aer_Nx.202003.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2020_03 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2020_03 = calculate_average_pm10(file_path_2020_03, shapefile_path_2020_03)
pm10_df_2020_03

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000007
3,WILL,0.000006
4,LASALLE,0.000006
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000007
100,UNION,0.000007


In [38]:
file_path_2021_01 = 'pm10-nasa/2021/MERRA2_400.tavgM_2d_aer_Nx.202101.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_01 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_01 = calculate_average_pm10(file_path_2021_01, shapefile_path_2021_01)
pm10_df_2021_01

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000003
3,WILL,0.000003
4,LASALLE,0.000003
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000003
100,UNION,0.000004


In [39]:
file_path_2021_02 = 'pm10-nasa/2021/MERRA2_400.tavgM_2d_aer_Nx.202102.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_02 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_02 = calculate_average_pm10(file_path_2021_02, shapefile_path_2021_02)
pm10_df_2021_02

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000003
3,WILL,0.000003
4,LASALLE,0.000003
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000003
100,UNION,0.000004


In [40]:
file_path_2021_03 = 'pm10-nasa/2021/MERRA2_400.tavgM_2d_aer_Nx.202103.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_03 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_03 = calculate_average_pm10(file_path_2021_03, shapefile_path_2021_03)
pm10_df_2021_03

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000008
3,WILL,0.000007
4,LASALLE,0.000008
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000008
100,UNION,0.000009


In [41]:
file_path_2021_04 = 'pm10-nasa/2021/MERRA2_400.tavgM_2d_aer_Nx.202104.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_04 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_04 = calculate_average_pm10(file_path_2021_04, shapefile_path_2021_04)
pm10_df_2021_04

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000011
3,WILL,0.000011
4,LASALLE,0.000011
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000012
100,UNION,0.000012


In [42]:
file_path_2021_05 = 'pm10-nasa/2021/MERRA2_400.tavgM_2d_aer_Nx.202105.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_05 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_05 = calculate_average_pm10(file_path_2021_05, shapefile_path_2021_05)
pm10_df_2021_05

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000006
3,WILL,0.000007
4,LASALLE,0.000006
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000008
100,UNION,0.000008


In [43]:
file_path_2021_06 = 'pm10-nasa/2021/MERRA2_401.tavgM_2d_aer_Nx.202106.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_06 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_06 = calculate_average_pm10(file_path_2021_06, shapefile_path_2021_06)
pm10_df_2021_06

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000008
3,WILL,0.000009
4,LASALLE,0.000009
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000010
100,UNION,0.000011


In [44]:
file_path_2021_07 = 'pm10-nasa/2021/MERRA2_401.tavgM_2d_aer_Nx.202107.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_07 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_07 = calculate_average_pm10(file_path_2021_07, shapefile_path_2021_07)
pm10_df_2021_07

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000054
3,WILL,0.000051
4,LASALLE,0.000052
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000053
100,UNION,0.000047


In [45]:
file_path_2021_08 = 'pm10-nasa/2021/MERRA2_401.tavgM_2d_aer_Nx.202108.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_08 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_08 = calculate_average_pm10(file_path_2021_08, shapefile_path_2021_08)
pm10_df_2021_08

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000030
3,WILL,0.000027
4,LASALLE,0.000028
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000025
100,UNION,0.000023


In [46]:
file_path_2021_09 = 'pm10-nasa/2021/MERRA2_401.tavgM_2d_aer_Nx.202109.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_09 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_09 = calculate_average_pm10(file_path_2021_09, shapefile_path_2021_09)
pm10_df_2021_09

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000023
3,WILL,0.000024
4,LASALLE,0.000024
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000025
100,UNION,0.000028


In [47]:
file_path_2021_10 = 'pm10-nasa/2021/MERRA2_401.tavgM_2d_aer_Nx.202110.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_10 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_10 = calculate_average_pm10(file_path_2021_10, shapefile_path_2021_10)
pm10_df_2021_10

Error: File not found at 'pm10-nasa/2021/MERRA2_401.tavgM_2d_aer_Nx.202110.nc4' or 'illinois_shape/IL_BNDY_County_Py.shp'


In [48]:
file_path_2021_11 = 'pm10-nasa/2021/MERRA2_400.tavgM_2d_aer_Nx.202111.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_11 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_11 = calculate_average_pm10(file_path_2021_11, shapefile_path_2021_11)
pm10_df_2021_11

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000004
3,WILL,0.000005
4,LASALLE,0.000004
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000005
100,UNION,0.000005


In [49]:
file_path_2021_12 = 'pm10-nasa/2021/MERRA2_400.tavgM_2d_aer_Nx.202112.nc4'
# Replace 'path/to/your/illinois_counties.shp' with the actual path to your shapefile
shapefile_path_2021_12 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2021_12 = calculate_average_pm10(file_path_2021_12, shapefile_path_2021_12)
pm10_df_2021_12

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')


,County,Average PM10
0,MCHENRY,0.000000
1,BOONE,0.000000
2,OGLE,0.000003
3,WILL,0.000003
4,LASALLE,0.000003
...,...,...
97,JEFFERSON,0.000000
98,LAWRENCE,0.000000
99,MARION,0.000003
100,UNION,0.000003


Combine the 2021 monthly data together to find the averages

In [56]:
list_of_dataframes = [pm10_df_2021_01, pm10_df_2021_02, pm10_df_2021_03, pm10_df_2021_04, pm10_df_2021_05, pm10_df_2021_06, pm10_df_2021_07, pm10_df_2021_08, pm10_df_2021_09, pm10_df_2021_10, pm10_df_2021_11, pm10_df_2021_12]
pm10_2021_concat = pd.concat(list_of_dataframes, ignore_index=True)
pm10_2021_mean = pm10_2021_concat.groupby('County')['Average PM10'].mean().reset_index()
pm10_2021_mean
pm10_2021_mean.to_csv('pm10_2021_mean.csv', index=False)

In [51]:
file_path_2022_01 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202201.nc4'
shapefile_path_2022_01 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_01 = calculate_average_pm10(file_path_2022_01, shapefile_path_2022_01)

file_path_2022_02 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202202.nc4'
shapefile_path_2022_02 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_02 = calculate_average_pm10(file_path_2022_02, shapefile_path_2022_02)

file_path_2022_03 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202203.nc4'
shapefile_path_2022_03 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_03 = calculate_average_pm10(file_path_2022_03, shapefile_path_2022_03)

file_path_2022_04 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202204.nc4'
shapefile_path_2022_04 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_04 = calculate_average_pm10(file_path_2022_04, shapefile_path_2022_04)

file_path_2022_05 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202205.nc4'
shapefile_path_2022_05 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_05 = calculate_average_pm10(file_path_2022_05, shapefile_path_2022_05)

file_path_2022_06 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202206.nc4'
shapefile_path_2022_06 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_06 = calculate_average_pm10(file_path_2022_06, shapefile_path_2022_06)

file_path_2022_07 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202207.nc4'
shapefile_path_2022_07 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_07 = calculate_average_pm10(file_path_2022_07, shapefile_path_2022_07)

file_path_2022_08 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202208.nc4'
shapefile_path_2022_08 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_08 = calculate_average_pm10(file_path_2022_08, shapefile_path_2022_08)

file_path_2022_09 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202209.nc4'
shapefile_path_2022_09 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_09 = calculate_average_pm10(file_path_2022_09, shapefile_path_2022_09)

file_path_2022_10 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202210.nc4'
shapefile_path_2022_10 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_10 = calculate_average_pm10(file_path_2022_10, shapefile_path_2022_10)

file_path_2022_11 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202211.nc4'
shapefile_path_2022_11 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_11 = calculate_average_pm10(file_path_2022_11, shapefile_path_2022_11)

file_path_2022_12 = 'pm10-nasa/2022/MERRA2_400.tavgM_2d_aer_Nx.202212.nc4'
shapefile_path_2022_12 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2022_12 = calculate_average_pm10(file_path_2022_12, shapefile_path_2022_12)


Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NA

In [55]:
list_of_dataframes = [pm10_df_2022_01, pm10_df_2022_02, pm10_df_2022_03, pm10_df_2022_04, pm10_df_2022_05, pm10_df_2022_06, pm10_df_2022_07, pm10_df_2022_08, pm10_df_2022_09, pm10_df_2022_10, pm10_df_2022_11, pm10_df_2022_12]
pm10_2022_concat = pd.concat(list_of_dataframes, ignore_index=True)
pm10_2022_mean = pm10_2022_concat.groupby('County')['Average PM10'].mean().reset_index()
pm10_2022_mean
pm10_2022_mean.to_csv('pm10_2022_mean.csv', index=False)

In [53]:
file_path_2023_01 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202301.nc4'
shapefile_path_2023_01 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_01 = calculate_average_pm10(file_path_2023_01, shapefile_path_2023_01)

file_path_2023_02 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202302.nc4'
shapefile_path_2023_02 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_02 = calculate_average_pm10(file_path_2023_02, shapefile_path_2023_02)

file_path_2023_03 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202303.nc4'
shapefile_path_2023_03 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_03 = calculate_average_pm10(file_path_2023_03, shapefile_path_2023_03)

file_path_2023_04 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202304.nc4'
shapefile_path_2023_04 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_04 = calculate_average_pm10(file_path_2023_04, shapefile_path_2023_04)

file_path_2023_05 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202305.nc4'
shapefile_path_2023_05 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_05 = calculate_average_pm10(file_path_2023_05, shapefile_path_2023_05)

file_path_2023_06 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202306.nc4'
shapefile_path_2023_06 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_06 = calculate_average_pm10(file_path_2023_06, shapefile_path_2023_06)

file_path_2023_07 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202307.nc4'
shapefile_path_2023_07 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_07 = calculate_average_pm10(file_path_2023_07, shapefile_path_2023_07)

file_path_2023_08 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202308.nc4'
shapefile_path_2023_08 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_08 = calculate_average_pm10(file_path_2023_08, shapefile_path_2023_08)

file_path_2023_09 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202309.nc4'
shapefile_path_2023_09 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_09 = calculate_average_pm10(file_path_2023_09, shapefile_path_2023_09)

file_path_2023_10 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202310.nc4'
shapefile_path_2023_10 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_10 = calculate_average_pm10(file_path_2023_10, shapefile_path_2023_10)

file_path_2023_11 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202311.nc4'
shapefile_path_2023_11 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_11 = calculate_average_pm10(file_path_2023_11, shapefile_path_2023_11)

file_path_2023_12 = 'pm10-nasa/2023/MERRA2_400.tavgM_2d_aer_Nx.202312.nc4'
shapefile_path_2023_12 = 'illinois_shape/IL_BNDY_County_Py.shp'
pm10_df_2023_12 = calculate_average_pm10(file_path_2023_12, shapefile_path_2023_12)

Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NAM', 'CO_FIPS', 'geometry'], dtype='object')
Columns in the shapefile:
Index(['COUNTY_NA

In [54]:
list_of_dataframes = [pm10_df_2023_01, pm10_df_2023_02, pm10_df_2023_03, pm10_df_2023_04, pm10_df_2023_05, pm10_df_2023_06, pm10_df_2023_07, pm10_df_2023_08, pm10_df_2023_09, pm10_df_2023_10, pm10_df_2023_11, pm10_df_2023_12]
pm10_2023_concat = pd.concat(list_of_dataframes, ignore_index=True)
pm10_2023_mean = pm10_2023_concat.groupby('County')['Average PM10'].mean().reset_index()
pm10_2023_mean
pm10_2023_mean.to_csv('pm10_2023_mean.csv', index=False)